<a href="https://colab.research.google.com/github/choki0715/lecture/blob/master/MLP_MNIST_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np


# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# 2개의 은닉층을 가진 MLP 모델 정의
class MLP(nn.Module):
    def __init__(self, input_size=784, hidden1_size=512, hidden2_size=256, num_classes=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden2_size, num_classes)

    def forward(self, x):
        # 입력 이미지를 1차원으로 펼치기
        x = x.view(x.size(0), -1)
        # 첫 번째 은닉층
        x = self.fc1(x)
        x = self.relu1(x)
        # 두 번째 은닉층
        x = self.fc2(x)
        x = self.relu2(x)
        # 출력층
        x = self.fc3(x)
        return x

# 하이퍼파라미터 설정
batch_size = 128
learning_rate = 0.001
num_epochs = 10

# 데이터 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST 평균과 표준편차
])

# MNIST 데이터셋 로드
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# 모델 초기화
model = MLP().to(device)
print(model)
print(f'\n총 파라미터 수: {sum(p.numel() for p in model.parameters()):,}')


Using device: cpu


100%|██████████| 9.91M/9.91M [00:00<00:00, 31.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.06MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 9.78MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 17.6MB/s]

MLP(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)

총 파라미터 수: 535,818


In [ ]:

# 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 함수
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        # 기울기 초기화
        optimizer.zero_grad()

        # 순전파
        output = model(data)
        loss = criterion(output, target)

        # 역전파
        loss.backward()
        optimizer.step()

        # 통계 계산
        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        if batch_idx % 100 == 0:
            print(f'Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

    accuracy = 100. * correct / total
    avg_loss = train_loss / len(train_loader)
    return avg_loss, accuracy

# 테스트 함수
def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

    test_loss /= len(test_loader)
    accuracy = 100. * correct / total

    print(f'\nTest set: Average loss: {test_loss:.4f}, '
          f'Accuracy: {correct}/{total} ({accuracy:.2f}%)\n')

    return test_loss, accuracy

# 학습 실행
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

print('학습 시작...\n')
for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train(model, device, train_loader, optimizer, criterion, epoch)
    test_loss, test_acc = test(model, device, test_loader, criterion)

    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)


학습 시작...

Epoch: 1 [0/60000 (0%)]	Loss: 2.295684
Epoch: 1 [12800/60000 (21%)]	Loss: 0.226062
Epoch: 1 [25600/60000 (43%)]	Loss: 0.222512
Epoch: 1 [38400/60000 (64%)]	Loss: 0.117284
Epoch: 1 [51200/60000 (85%)]	Loss: 0.095173

Test set: Average loss: 0.1085, Accuracy: 9646/10000 (96.46%)

Epoch: 2 [0/60000 (0%)]	Loss: 0.061509
Epoch: 2 [12800/60000 (21%)]	Loss: 0.022178
Epoch: 2 [25600/60000 (43%)]	Loss: 0.042374
Epoch: 2 [38400/60000 (64%)]	Loss: 0.036901
Epoch: 2 [51200/60000 (85%)]	Loss: 0.078590

Test set: Average loss: 0.0972, Accuracy: 9705/10000 (97.05%)

Epoch: 3 [0/60000 (0%)]	Loss: 0.079712
Epoch: 3 [12800/60000 (21%)]	Loss: 0.047117
Epoch: 3 [25600/60000 (43%)]	Loss: 0.017512
Epoch: 3 [38400/60000 (64%)]	Loss: 0.020257
Epoch: 3 [51200/60000 (85%)]	Loss: 0.035660

Test set: Average loss: 0.0720, Accuracy: 9768/10000 (97.68%)

Epoch: 4 [0/60000 (0%)]	Loss: 0.044682
Epoch: 4 [12800/60000 (21%)]	Loss: 0.079537
Epoch: 4 [25600/60000 (43%)]	Loss: 0.018016
Epoch: 4 [38400/60000 (64%

In [ ]:

# 결과 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# 손실 그래프
ax1.plot(range(1, num_epochs + 1), train_losses, label='Train Loss', marker='o')
ax1.plot(range(1, num_epochs + 1), test_losses, label='Test Loss', marker='s')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Test Loss')
ax1.legend()
ax1.grid(True)

# 정확도 그래프
ax2.plot(range(1, num_epochs + 1), train_accuracies, label='Train Accuracy', marker='o')
ax2.plot(range(1, num_epochs + 1), test_accuracies, label='Test Accuracy', marker='s')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Training and Test Accuracy')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/training_results.png', dpi=300, bbox_inches='tight')
print('학습 결과 그래프가 저장되었습니다.')

# 모델 저장
torch.save(model.state_dict(), '/mnt/user-data/outputs/mnist_mlp.pth')
print('모델이 저장되었습니다.')

# 예측 샘플 시각화
model.eval()
with torch.no_grad():
    # 테스트 데이터에서 샘플 추출
    data_iter = iter(test_loader)
    images, labels = next(data_iter)
    images, labels = images.to(device), labels.to(device)

    # 예측
    outputs = model(images)
    _, predicted = torch.max(outputs, 1)

    # 처음 10개 샘플 시각화
    fig, axes = plt.subplots(2, 5, figsize=(12, 5))
    for idx, ax in enumerate(axes.flat):
        img = images[idx].cpu().squeeze()
        ax.imshow(img, cmap='gray')
        ax.set_title(f'예측: {predicted[idx].item()}\n실제: {labels[idx].item()}',
                    color='green' if predicted[idx] == labels[idx] else 'red')
        ax.axis('off')

    plt.tight_layout()
    plt.savefig('/mnt/user-data/outputs/predictions.png', dpi=300, bbox_inches='tight')
    print('예측 결과 시각화가 저장되었습니다.')

print(f'\n최종 테스트 정확도: {test_accuracies[-1]:.2f}%')

In [ ]:
# ============================================================
# 예측 결과 시각화 (요청하신 코드 적용)
# ============================================================
print("\n" + "="*60)
print("예측 결과 시각화 중...")
print("="*60)

model.eval()
dataiter = iter(test_loader)
images, labels = next(dataiter)
images = images.to(device)  # Move images to the device

# get sample outputs
output = model(images)
# convert output probabilities to predicted class
_, preds = torch.max(output, 1)
# prep images for display
images = images.cpu().numpy()  # Move images back to CPU for numpy conversion

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(16, 16))
for idx in np.arange(36):
    ax = fig.add_subplot(6, 6, idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    ax.set_title("{} ({})".format(str(preds[idx].item()), str(labels[idx].item())),
                 color=("green" if preds[idx]==labels[idx] else "red"),
                 fontsize=10, fontweight='bold')

plt.suptitle('Prediction Results (Predicted (True Label))\nGreen: Correct, Red: Incorrect',
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/predictions_visualization.png', dpi=150, bbox_inches='tight')
print("예측 결과 시각화가 저장되었습니다: predictions_visualization.png")

# 정확도 통계
correct_predictions = (preds == labels.to(device)).sum().item()
total_predictions = len(labels)
accuracy_batch = 100. * correct_predictions / total_predictions
print(f"\n이 배치의 정확도: {correct_predictions}/{total_predictions} = {accuracy_batch:.2f}%")

# 오분류된 샘플 분석
misclassified = []
for idx in range(len(preds)):
    if preds[idx] != labels[idx]:
        misclassified.append({
            'index': idx,
            'predicted': preds[idx].item(),
            'true': labels[idx].item()
        })

if misclassified:
    print(f"\n오분류된 샘플: {len(misclassified)}개")
    print("처음 5개의 오분류:")
    for i, mis in enumerate(misclassified[:5]):
        print(f"  {i+1}. 인덱스 {mis['index']}: 예측={mis['predicted']}, 실제={mis['true']}")
else:
    print("\n이 배치에서 모든 예측이 정확합니다! 🎉")

# 혼동 행렬 생성 (전체 테스트 데이터)
print("\n" + "="*60)
print("전체 테스트 데이터에 대한 혼동 행렬 생성 중...")
print("="*60)

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for data, target in test_loader:
        data = data.to(device)
        output = model(data)
        _, predicted = torch.max(output, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(target.numpy())

# 혼동 행렬
cm = confusion_matrix(all_labels, all_preds)

# 혼동 행렬 시각화
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=range(10), yticklabels=range(10))
plt.title('Confusion Matrix', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig('/mnt/user-data/outputs/confusion_matrix.png', dpi=150, bbox_inches='tight')
print("혼동 행렬이 저장되었습니다: confusion_matrix.png")

# 분류 리포트
print("\n분류 리포트:")
print(classification_report(all_labels, all_preds,
                          target_names=[str(i) for i in range(10)]))

# 모델 저장
torch.save(model.state_dict(), '/mnt/user-data/outputs/mnist_cnn_model.pth')
print("\n" + "="*60)
print("모델이 저장되었습니다: mnist_cnn_model.pth")
print("="*60)

print("\n✅ 모든 작업이 완료되었습니다!")
print("\n생성된 파일:")
print("  1. training_history.png - 학습 곡선")
print("  2. predictions_visualization.png - 36개 샘플 예측 결과")
print("  3. confusion_matrix.png - 혼동 행렬")
print("  4. mnist_cnn_model.pth - 학습된 모델")